In [1]:
# token e11620f2f2e953c08574f9a82e5a94b0c24cf77c36c958f7

In [2]:
import mlflow
import pandas as pd
from mlflow import tracking
from mlflow.entities import ViewType
import yaml
import json
import numpy as np
import sys  # Import the sys module for stderr
import os
import time

In [3]:
# Define a custom function to convert NumPy objects to standard Python objects
def numpy_to_python(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, np.generic):
        return np.asscalar(obj)
    return obj

# Set the MLflow tracking server URI
#tracking_uri = "http://127.0.0.1:5000"  # Replace with your MLflow server URI
tracking_uri = os.environ["MLFLOW_TRACKING_URI"]  # Replace with your MLflow server URI
print(f"connecting with: {tracking_uri}", file=sys.stderr)
tracking.set_tracking_uri(tracking_uri)

connecting with: http://server:5000


In [4]:
# Retrieve all experiments and create a dictionary to map experiment IDs to names
print(f"searching for experiments", file=sys.stderr)
experiment_id_to_name = {}
for exp in mlflow.search_experiments():
    if "_test" not in exp.name:
        print(f" - found {exp.name}", file=sys.stderr)
        experiment_id_to_name[exp.experiment_id] = exp.name

# Retrieve all experiments
all_experiments = list(experiment_id_to_name.keys())

searching for experiments
 - found unconditional-benchmark-POWER
 - found unconditional-benchmark-MINIBOONE
 - found unconditional-benchmark-HEPMASS
 - found unconditional-benchmark-BSDS300
 - found unconditional-hybrid-pre-trained-circles
 - found unconditional-hybrid-pre-trained-moons
 - found unconditional-hybrid-circles
 - found unconditional-hybrid-moons
 - found conditional-circles
 - found conditional-moons
 - found unconditional-circles
 - found unconditional-moons
 - found unconditional-moons


In [5]:
#list(experiment_id_to_name.items())[12]

In [6]:
ts = time.time()
print("downloading runs", file=sys.stderr)
all_runs = []
progress_counter = 0
for experiment_id in all_experiments:
    try:
        runs = mlflow.search_runs(
            experiment_ids=[experiment_id],
            run_view_type=ViewType.ALL,
            max_results=1000000
        )
        all_runs.append(runs)
        progress_counter += 1
        print(f"Downloaded {progress_counter}/{len(all_experiments)} experiments", file=sys.stderr)
        time.sleep(1) # to avoid to many requests error (and speed up download time??)
    except Exception as e:
        all_runs.append(None)
        progress_counter += 1
        print(f"skipping {progress_counter} {list(experiment_id_to_name.items())[progress_counter-1]}", file=sys.stderr)
        
te = time.time()
te-ts

downloading runs
Downloaded 1/13 experiments
Downloaded 2/13 experiments
Downloaded 3/13 experiments
Downloaded 4/13 experiments
Downloaded 5/13 experiments
Downloaded 6/13 experiments
Downloaded 7/13 experiments
Downloaded 8/13 experiments
Downloaded 9/13 experiments
Downloaded 10/13 experiments
Downloaded 11/13 experiments
Downloaded 12/13 experiments
Downloaded 13/13 experiments


611.9529712200165

In [7]:
#all_runs[15]
#experiment_id_to_name[all_experiments[14]]

In [8]:
df = pd.concat(all_runs)
df["experiment_name"] = df.apply(lambda row: experiment_id_to_name[row["experiment_id"]], axis=1)

df = df.dropna(axis=1, how='all')
df = df.sort_values(["metrics.val_loss"])

for c in df.columns:
    print(c)

run_id
experiment_id
status
artifact_uri
start_time
end_time
metrics.val_loss
metrics.min_loss
metrics.final_epoch
metrics.lr
metrics.best_epoch
metrics.min_val_loss
metrics.loss
params.after_fit_hook
params.fit_kwds.batch_size
params.model_kwds.distribution_kwds.shift
params.min_delta
params.results_path
params.model_kwds.distribution_kwds.scale
params.model_kwds.distribution_kwds.clip_to_bernstein_domain
params.monitor
params.model_kwds.distribution_kwds.base_distribution_kwds.distribution_type
params.initial_epoch
params.model_kwds.parameter_kwds.hidden_units
params.model_kwds.distribution_kwds.allow_flexible_bounds
params.model_kwds.distribution
params.model_kwds.distribution_kwds.smooth_bounds
params.fit_kwds.lr_patience
params.fit_kwds.epochs
params.model_kwds.distribution_kwds.base_distribution_kwds.concentration0
params.seed
params.validation_batch_size
params.fit_kwds.verbose
params.epochs
params.fit_kwds.reduce_lr_on_plateau
params.class_weight
params.restore_best_weights
par

/tmp/ipykernel_253/462501590.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["experiment_name"] = df.apply(lambda row: experiment_id_to_name[row["experiment_id"]], axis=1)


In [9]:
# clean run names for easy aggregation and comparison later on
def remove_exp_from_run_name(row):
    exp_name = row["experiment_name"]
    words = exp_name.split("-")

    # Remove the last word by slicing the list of words
    words_without_last = words[:-1]

    # Join the remaining words back into a single string using "-" as the delimiter
    exp_name = "_".join(words_without_last)
    
    run_name = row["tags.mlflow.runName"]
    new_name = run_name.replace(f"{exp_name}_", "")
    return new_name

df["tags.mlflow.runName"] = df.apply(remove_exp_from_run_name, axis=1)

In [23]:
q1 = df["params."+"fit_kwds.learning_rate"] == "0.0001"
q2 = df["experiment_name"] == "unconditional-benchmark-POWER"
q3 = df["tags.mlflow.runName"] == "bernstein_flow"
qs = [q1, q2, q3]
df[np.logical_and.reduce(qs)].shape

(12, 111)

In [12]:
df.pivot_table(values=df, index=["experiment_name", "tags.mlflow.runName"], aggfunc="first")

artifact_uri  \
experiment_name                          tags.mlflow.runName                                                                                     
conditional-circles                      bernstein_flow                                      file:///mlruns/788739528573814156/05eed33dc2a0...   
                                         coupling_bernstein_flow                             file:///mlruns/788739528573814156/a3af65aba3d9...   
                                         masked_autoregressive_bernstein_flow                file:///mlruns/788739528573814156/bde2919f6ed8...   
                                         multivariate_bernstein_flow                         file:///mlruns/788739528573814156/c9731480e491...   
                                         multivariate_normal                                 file:///mlruns/788739528573814156/da9d647d4c25...   
conditional-moons                        bernstein_flow                                      file:///mlruns/117130953417680787/4ac3ce0a3766...   
                                         coupling_bernstein_flow                             file:///mlruns/117130953417680787/87541df97f27...   
                                         masked_autoregressive_bernstein_flow                file:///mlruns/117130953417680787/55d8cf321e6f...   
                                         multivariate_bernstein_flow                         file:///mlruns/117130953417680787/50bbdc3d859d...   
                                         multivariate_normal                                 file:///mlruns/117130953417680787/e095ebf299f1...   
unconditional-benchmark-BSDS300          bernstein_flow                                      file:///mlruns/423080604963110590/61c96050617b...   
                                         masked_autoregressive_bernstein_flow                file:///mlruns/423080604963110590/0817877460df...   
unconditional-benchmark-HEPMASS          bernstein_flow                                      file:///mlruns/790289923366103498/57dc69ce01aa...   
                                         masked_autoregressive_bernstein_flow                file:///mlruns/790289923366103498/2769a9a00f49...   
unconditional-benchmark-MINIBOONE        bernstein_flow                                      file:///mlruns/368702643869141648/d1c851d44898...   
                                         masked_autoregressive_bernstein_flow                file:///mlruns/368702643869141648/588aab9196e6...   
unconditional-benchmark-POWER            bernstein_flow                                      file:///mlruns/847109317117479321/b2b52873f989...   
                                         masked_autoregressive_bernstein_flow                file:///mlruns/847109317117479321/8623ad545ad7...   
unconditional-circles                    bernstein_flow                                      file:///mlruns/820079630308701613/37a7dd6a28a6...   
                                         coupling_bernstein_flow                             file:///mlruns/820079630308701613/cd1672471f21...   
                                         masked_autoregressive_bernstein_flow                file:///mlruns/820079630308701613/42b4cb04444b...   
                                         multivariate_bernstein_flow                         file:///mlruns/820079630308701613/af0810eceaaf...   
                                         multivariate_normal                                 file:///mlruns/820079630308701613/68234176a1e6...   
unconditional-hybrid-circles             coupling_bernstein_flow                             file:///mlruns/123580947345058918/3a7bbd965b2b...   
                                         masked_autoregressive_bernstein_flow_first_dim_...  file:///mlruns/123580947345058918/849318cc3b92...   
unconditional-hybrid-moons               coupling_bernstein_flow                             file:///mlruns/433784438971719645/3df6600fc61d...   
                                         m

## Generate summarizing dataframe

In [ ]:
grouped_df  = df.groupby(["experiment_name", "tags.mlflow.runName"])

# dipslay
grouped_df.apply(lambda r: r[:])

### Best validation losses


 vgl mit arxiv.org/abs/2102.03509

In [ ]:
idx = grouped_df["metrics.val_loss"].idxmin()
df_best_runs = df.loc[idx]
pd.pivot_table(df_best_runs[["experiment_name", 
                             "tags.mlflow.runName", 
                             "metrics.val_loss", 
                             ]], values="metrics.val_loss", index="experiment_name", columns=["tags.mlflow.runName"])

### Best N by experiment alone

In [ ]:
df_best_n = df
df_best_n["rank"] = df_best_n.groupby("experiment_name")["metrics.val_loss"].rank(ascending=True, method="first").replace(np.nan, 1.0).astype(int)

N_BEST_RUNS = 3
df_best_n = df_best_n[df_best_n["rank"] <= N_BEST_RUNS]
df_best_n.pivot_table(values=df_best_n[["run_id","tags.mlflow.runName", "metrics.val_loss"]], index=["experiment_name","rank"], aggfunc="first")
#TODO: veränderte Parameter anzeigen

### Best N runs for each experiment by runName

In [ ]:
df_best_n = df
df_best_n["rank"] = grouped_df["metrics.val_loss"].rank(ascending=True, method="first").replace(np.nan, 1.0).astype(int)

N_BEST_RUNS = 3
df_best_n = df_best_n[df_best_n["rank"] <= N_BEST_RUNS]
pd.options.display.max_rows = 100
table = df_best_n.pivot_table(values=df_best_n[["run_id", "metrics.val_loss","params.fit_kwds.batch_size", 
                             "params.fit_kwds.learning_rate", 
                             "params.fit_kwds.lr_patience", 
                             "params.model_kwds.distribution_kwds.order", 
                             "params.model_kwds.parameter_kwds.hidden_units"]], index=["experiment_name", "tags.mlflow.runName","rank"], aggfunc="first")
table

In [ ]:
table.to_excel("table.xlsx")